In [3]:
import requests
import json
import pandas as pd
import re
import numpy as np

In [40]:
test = pd.DataFrame({'Address': ['414 Grant Street, Pittsburgh PA', '2316 Patterson Street, Pittsbugh PA 15203']})

address = '414 Grant Street, Pittsburgh PA'

In [61]:
def county_geocode (address):
    url = "http://gisdata.alleghenycounty.us/arcgis/rest/services/Geocoders/Composite/GeocodeServer/findAddressCandidates"

    query = {"SingleLine": address ,"outSR":"4326","f":"pjson"}
    r = requests.request("GET", url, params=query)
    j = r.json()['candidates']
    c = pd.DataFrame.from_dict(j)
    
    if c.shape[0] > 0:
        top = c.head(1)
        locs = str(top['location'])
        x = re.search(': (.+?),', locs).group(1)
        y = re.search('\'y\': (.+?)}', locs).group(1)
        df = pd.DataFrame({'X': [x], 'Y': [y]})
    else:
        df = pd.DataFrame({'Address': [address], 'X': [np.nan], 'Y': [np.nan]})
        
    return(df)

In [90]:
def county_geo_pd (df, col = 'Address'):
    final = pd.DataFrame({'X' : [], 'Y' : []})
    
    for index, row in test.iterrows():
        temp = county_geocode(row['Address'])
        final = final.append(temp)
    
    return(pd.concat([df, final.reset_index(drop=True)], axis = 1))

In [91]:
county_geo_pd(test)

,Address,X,Y
0,"414 Grant Street, Pittsburgh PA",-79.99685950610291,40.43811465835781
1,"2316 Patterson Street, Pittsbugh PA 15203",-79.9733880847145,40.41927085425008
